In [1]:
import openai
import os
import pandas as pd
import tweepy

consumer_key = "BkEPb0jyEMmw878gj74vOrkFY"
consumer_secret = "24dZiQhAZdTX4QFuZc9EZpfYJdnw83tzTARVq9KH0mITjQrcyE"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAL4NnAEAAAAAp%2BpBIxt%2FjZja7nn5wEZtInIgCAs%3DFzdsKA1nkcN6OrdDlCGiuKHcpRDowA8lcNAmZT0FVwl5YefguE"

auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
twitterAPI = tweepy.API(auth)

ModuleNotFoundError: No module named 'tweepy'

In [2]:
tweet_dataset = []
tweets = twitterAPI.search_tweets(q='RDNT -filter:retweets -filter:replies',
                                  lang = "en",
                                  result_type="recent",
                                  count = 10
                                 )

for tweet in tweets:
    tweet_content = {'id': tweet.id,
                     'date': tweet.created_at,
                     'user_name': tweet.user.screen_name,
                     'text': tweet.text,
                     }

    tweet_dataset.append(tweet_content)

tweet_dataframe = pd.DataFrame(tweet_dataset)
print(tweet_dataframe.shape)
tweet_dataframe.head()


(10, 4)


,id,date,user_name,text
0,1651355063938592768,2023-04-26 22:38:22+00:00,Banstok2,"Guys, Don't miss the next move in a few hours:..."
1,1651355017738346496,2023-04-26 22:38:11+00:00,Banstok2,"Guys, Don't miss the next move in a few hours:..."
2,1651354995911200768,2023-04-26 22:38:06+00:00,Banstok2,"Guys, Don't miss the next move in a few hours:..."
3,1651354937861996544,2023-04-26 22:37:52+00:00,Banstok2,"Guys, Don't miss the next move in a few hours:..."
4,1651354935810981888,2023-04-26 22:37:52+00:00,BenjaminMilo4,ARE YOU READY for NEXT BULLISH TRADE🚀 $FTM $ST...
